IMPORTS

In [21]:
from ortools.linear_solver import pywraplp

In [22]:
# valores -> lista dos tamanhos de cada tipo de item
# valor_maximo -> tamanho da barra (a soma das combinações não pode ultrapassar)
# menor_valor -> menor tamanho da lista "valores" -> valor da restrição 
# combinacao_atual-> uma lista com as combinação de itens atuais
# inicio -> indice de inicio
# resultados -> lista vazia que armazenará todos os resultados válidos 

# Função que calcula as combinações válidas (a restrição de soma máxima e a diferença mínima)
def explorar_combinacoes(valores, valor_maximo, menor_valor, combinacao_atual, inicio, resultados):

    soma_atual = sum(combinacao_atual)
    
    if soma_atual <= valor_maximo and (valor_maximo - soma_atual) < menor_valor:
        padrao = []
        # conta quantas vezes cada tamanho apareceu naquele padrão
        for valor in valores:
            padrao.append(combinacao_atual.count(valor))
        resultados.append(padrao)
    # se a soma ultrapassar o valor máximo, para e o padrão não é adicionado aos resultados   
    elif soma_atual > valor_maximo:
        return

    # se não, continuamos a adicionar elementos para formar novas combinações
    for i in range(inicio, len(valores)):
        combinacao_atual.append(valores[i])
        explorar_combinacoes(valores, valor_maximo, menor_valor, combinacao_atual, i, resultados)
        combinacao_atual.pop()  # Remove o último elemento para explorar outras combinações

In [23]:
# valores -> lista dos tamanhos de cada tipo de item
# valor_maximo -> tamanho da barra (a soma das combinações não pode ultrapassar)

# função gerdora das combinações válidas 
def encontrar_combinacoes_soma_maxima(valores, valor_maximo):

    resultados = []
    menor_valor = min(valores)  # menor tamanho da lista "valores" -> valor da restrição

    # calcula as combinações
    explorar_combinacoes(valores, valor_maximo, menor_valor, [], 0, resultados)
    return resultados

In [24]:
# comprimento_barra -> comprimento máximo da barra.
# tamanhos_itens -> lista de tamanhos de itens.
# quantidades_itens -> lista de quantidades demandadas de cada item.

# Função main
def minimizar_desperdicio(comprimento_barra, tamanhos_itens, quantidades_itens):
    
    # gera todos os padrões de corte válidos
    padroes = encontrar_combinacoes_soma_maxima(tamanhos_itens, comprimento_barra)

    # calculo do desperdício de cada padrão
    desperdicio_por_padrao = []
    for padrao in padroes:
        comprimento_utilizado = 0
        for i in range(len(tamanhos_itens)):
            comprimento_utilizado += padrao[i] * tamanhos_itens[i]
        desperdicio_por_padrao.append(comprimento_barra - comprimento_utilizado)

    # declara o solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # declara as varriáveis de decisão para cada padrão de corte
    variaveis_decisao = []
    for j in range(len(padroes)):
        variavel = solver.IntVar(0, solver.infinity(), f'x[{j}]')
        variaveis_decisao.append(variavel)

    # declara a função objetivo
    funcao_objetivo = 0
    for j in range(len(padroes)):
        funcao_objetivo += desperdicio_por_padrao[j] * variaveis_decisao[j]
    solver.Minimize(funcao_objetivo)

    # declara as restrições para atender à demanda de cada tipo de item
    for i in range(len(tamanhos_itens)):
        demanda = 0
        for j in range(len(padroes)):
            demanda += padroes[j][i] * variaveis_decisao[j]
        solver.Add(demanda >= quantidades_itens[i])

    # resolve o modelo
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:

        desperdicio_total = 0
        for j in range(len(padroes)):
            quantidade_usada = int(variaveis_decisao[j].solution_value())
            desperdicio_padrao = desperdicio_por_padrao[j] * quantidade_usada
            desperdicio_total += desperdicio_padrao
        print('Desperdício total: ', desperdicio_total)

        print('Padrões de corte utilizados:')
        for j in range(len(padroes)):
            quantidade_usada = int(variaveis_decisao[j].solution_value())
            if quantidade_usada > 0:
                print(f'Padrão {j + 1}: {padroes[j]}, usado {quantidade_usada} vezes')
        
    else:
        print('Modelo sem solução ótima.')

In [25]:
#  Coletando entradas do usuário
comprimento_barra = int(input("Informe o comprimento da barra: "))
quantidade_tipos_itens = int(input("Informe a quantidade de tipos de itens: "))

# Leitura dos tamanhos dos itens em uma única linha
tamanhos_itens = list(map(int, input("Informe os tamanhos dos itens, separados por espaço: ").split()))

# Leitura das quantidades necessárias em uma única linha
quantidades_itens = list(map(int, input("Informe as quantidades necessárias de cada item, separadas por espaço: ").split()))

# Valida o número de entradas
if len(tamanhos_itens) != quantidade_tipos_itens or len(quantidades_itens) != quantidade_tipos_itens:
    print("Erro: A quantidade de tipos de itens não corresponde ao número de tamanhos e quantidades informados.")
else:
    # Executa a função de minimização com as entradas do usuário
    minimizar_desperdicio(comprimento_barra, tamanhos_itens, quantidades_itens)


Desperdício total:  1000
Padrões de corte utilizados:
Padrão 1: [1, 1, 0], usado 100 vezes
Padrão 5: [0, 0, 3], usado 40 vezes
